In [24]:
import cv2
import numpy as np
import xlsxwriter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
import re

In [25]:
workbook = xlsxwriter.Workbook('D:/work/OutPutImages/files/COCO.xlsx')
worksheet1 = workbook.add_worksheet()


In [26]:
# Load Yolo

net = cv2.dnn.readNet("D:/work/yolov3.weights", "D:/AmirWork/yolov3.cfg")
classes = []
with open("D:/work/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
print(classes)
indx=0
for c in classes:
    worksheet1.write(indx+1, 1, str(indx))
    worksheet1.write(indx+1, 2, str(c))
    indx=indx+1
workbook.close()
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [27]:
print(output_layers)

['yolo_82', 'yolo_94', 'yolo_106']


In [28]:
import os
path = 'D:/work/images/'
dataset_path = os.listdir('D:/work/images/')
imageCount=1
imagesObjects=[]
workbook = xlsxwriter.Workbook('D:/work/OutPutImages/files/ImagesAnalysis.xlsx')
worksheet1 = workbook.add_worksheet()
workbookhwc = xlsxwriter.Workbook('D:/work/OutPutImages/files/hwc-ImagesAnalysis.xlsx')
worksheet1hwc = workbookhwc.add_worksheet()
row=0
col=0
worksheet1hwc.write(row, 0, str("Analysis of Image"))
worksheet1hwc.write(row, 1, str("Height of Image"))
worksheet1hwc.write(row, 2, str("Width of Image"))
worksheet1hwc.write(row, 3, str("Channels"))

worksheet1.write(row, 0, str("Analysis of Image"))
worksheet1.write(row, 1, str("Height of Image"))
worksheet1.write(row, 2, str("Width of Image"))
worksheet1.write(row, 3, str("Channels"))
worksheet1.write(row, 4, str("Detected Object"))
worksheet1.write(row, 5, str("Box size of detected object"))
worksheet1.write(row, 6, str("Confidence of detected object"))
worksheet1.write(row, 7, str("Class of detected object"))
worksheet1.write(row, 8, str("Name of detected object"))
row=1
row_hwc=1
for p in dataset_path:
    print("==========================Analyusis of Image:" + str(path + p))
    img = cv2.imread(path + p)
    img = cv2.resize(img, None, fx=0.9, fy=0.9)
    height, width, channels = img.shape
    worksheet1hwc.write(row_hwc, 0, str(path + p))
    worksheet1hwc.write(row_hwc, 1, str(height))
    worksheet1hwc.write(row_hwc, 2, str(width))
    worksheet1hwc.write(row_hwc, 3, str(channels))
    row_hwc=row_hwc+1
    worksheet1.write(row, 0, str(path + p))
    worksheet1.write(row, 1, str(height))
    worksheet1.write(row, 2, str(width))
    worksheet1.write(row, 3, str(channels))
    
    
    print("height of image: "+ str(height))
    print("width of image: "+ str(width))
    print("Channels of image are: " + str(channels))
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
            # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)    
    i=0
    row=row+1
    for b in boxes:
        print("Box size of detected object" + str(b))
        print("Confidence of detected object: " + str(confidences[i]))
        print("Class of detected object: " + str(class_ids[i]))
        print("Name of detected object" + str(classes[class_ids[i]]))
        print("..............................")
        worksheet1.write(row, 4, str("Object-" + str(i)))
        worksheet1.write(row, 5, str(b))
        worksheet1.write(row, 6, str(confidences[i]))
        worksheet1.write(row, 7, str(class_ids[i]))
        worksheet1.write(row, 8, str(classes[class_ids[i]]))
        i=i+1
        row=row+1
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    imageObjects=""
    for i in indexes:
        print(str(classes[class_ids[i]]))
        imageObjects+=str(classes[class_ids[i]]) + ","
    print(imageObjects)
    imagesObjects.append(imageObjects)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 3, color, 3)
    
    cv2.imwrite("D:/work/OutPutImages/" + str(p) +".jpg",img) 
    imageCount=imageCount+1
    row=row+1
workbook.close()
workbookhwc.close()


==========================Analyusis of Image:D:/AmirWork/images/image1.jpg
height of image: 608
width of image: 608
Channels of image are: 3
Box size of detected object[59, 55, 500, 510]
Confidence of detected object: 0.8877587914466858
Class of detected object: 75
Name of detected objectvase
..............................
Box size of detected object[90, 61, 484, 502]
Confidence of detected object: 0.6908643245697021
Class of detected object: 75
Name of detected objectvase
..............................
Box size of detected object[61, 86, 486, 491]
Confidence of detected object: 0.8369032740592957
Class of detected object: 75
Name of detected objectvase
..............................
Box size of detected object[93, 100, 483, 468]
Confidence of detected object: 0.5311260223388672
Class of detected object: 75
Name of detected objectvase
..............................
vase
vase,
==========================Analyusis of Image:D:/AmirWork/images/image10.jpg
height of image: 481
width of image


==========================Analyusis of Image:D:/AmirWork/images/image105.jpg
height of image: 411
width of image: 618
Channels of image are: 3
Box size of detected object[32, 116, 474, 247]
Confidence of detected object: 0.6011128425598145
Class of detected object: 8
Name of detected objectboat
..............................
Box size of detected object[34, 113, 539, 258]
Confidence of detected object: 0.5016247034072876
Class of detected object: 8
Name of detected objectboat
..............................
Box size of detected object[18, 141, 503, 242]
Confidence of detected object: 0.7328173518180847
Class of detected object: 8
Name of detected objectboat
..............................
boat
boat,
==========================Analyusis of Image:D:/AmirWork/images/image106.jpg
height of image: 384
width of image: 663
Channels of image are: 3
Box size of detected object[41, 4, 581, 274]
Confidence of detected object: 0.5811828374862671
Class of detected object: 7
Name of detected objecttruc

Box size of detected object[169, 35, 387, 233]
Confidence of detected object: 0.994969367980957
Class of detected object: 7
Name of detected objecttruck
..............................
Box size of detected object[166, 38, 394, 230]
Confidence of detected object: 0.9961962699890137
Class of detected object: 7
Name of detected objecttruck
..............................
Box size of detected object[211, 24, 347, 251]
Confidence of detected object: 0.6806191205978394
Class of detected object: 7
Name of detected objecttruck
..............................
Box size of detected object[178, 33, 415, 235]
Confidence of detected object: 0.9412334561347961
Class of detected object: 7
Name of detected objecttruck
..............................
Box size of detected object[168, 51, 387, 221]
Confidence of detected object: 0.990836501121521
Class of detected object: 7
Name of detected objecttruck
..............................
Box size of detected object[160, 44, 405, 233]
Confidence of detected object:

Box size of detected object[41, 55, 409, 389]
Confidence of detected object: 0.9519966244697571
Class of detected object: 7
Name of detected objecttruck
..............................
Box size of detected object[44, 150, 401, 274]
Confidence of detected object: 0.9213677644729614
Class of detected object: 7
Name of detected objecttruck
..............................
Box size of detected object[38, 198, 415, 239]
Confidence of detected object: 0.5313539505004883
Class of detected object: 7
Name of detected objecttruck
..............................
truck
truck,
==========================Analyusis of Image:D:/AmirWork/images/image119.jpg
height of image: 348
width of image: 731
Channels of image are: 3
Box size of detected object[186, 25, 491, 264]
Confidence of detected object: 0.9578270316123962
Class of detected object: 7
Name of detected objecttruck
..............................
Box size of detected object[183, 26, 502, 288]
Confidence of detected object: 0.9958776235580444
Class of

height of image: 436
width of image: 582
Channels of image are: 3
Box size of detected object[-1, 195, 88, 29]
Confidence of detected object: 0.9652389287948608
Class of detected object: 2
Name of detected objectcar
..............................
Box size of detected object[0, 195, 92, 29]
Confidence of detected object: 0.9720159769058228
Class of detected object: 2
Name of detected objectcar
..............................
Box size of detected object[112, 199, 86, 26]
Confidence of detected object: 0.9744054675102234
Class of detected object: 2
Name of detected objectcar
..............................
Box size of detected object[116, 199, 86, 26]
Confidence of detected object: 0.9847464561462402
Class of detected object: 2
Name of detected objectcar
..............................
Box size of detected object[335, 194, 77, 31]
Confidence of detected object: 0.6286768317222595
Class of detected object: 2
Name of detected objectcar
..............................
Box size of detected object

...

Box size of detected object[9, 33, 312, 334]
Confidence of detected object: 0.8598305583000183
Class of detected object: 50
Name of detected objectbroccoli
..............................
Box size of detected object[298, 21, 341, 341]
Confidence of detected object: 0.5062922835350037
Class of detected object: 50
Name of detected objectbroccoli
..............................
broccoli
broccoli
broccoli,broccoli,
==========================Analyusis of Image:D:/AmirWork/images/image63.jpg
height of image: 495
width of image: 515
Channels of image are: 3
Box size of detected object[20, -14, 411, 392]
Confidence of detected object: 0.7556464672088623
Class of detected object: 50
Name of detected objectbroccoli
..............................
Box size of detected object[18, -20, 469, 407]
Confidence of detected object: 0.8036277890205383
Class of detected object: 50
Name of detected objectbroccoli
..............................
Box size of detected object[11, -3, 432, 443]
Confidence of detecte

height of image: 770
width of image: 478
Channels of image are: 3
Box size of detected object[175, 29, 120, 248]
Confidence of detected object: 0.9678663015365601
Class of detected object: 9
Name of detected objecttraffic light
..............................
Box size of detected object[167, 9, 131, 310]
Confidence of detected object: 0.9887798428535461
Class of detected object: 9
Name of detected objecttraffic light
..............................
Box size of detected object[165, 33, 136, 302]
Confidence of detected object: 0.7344526648521423
Class of detected object: 9
Name of detected objecttraffic light
..............................
traffic light
traffic light,
==========================Analyusis of Image:D:/AmirWork/images/image70.jpg
height of image: 639
width of image: 400
Channels of image are: 3
Box size of detected object[191, 283, 41, 54]
Confidence of detected object: 0.5090897679328918
Class of detected object: 9
Name of detected objecttraffic light
........................

Box size of detected object[37, 75, 504, 306]
Confidence of detected object: 0.7837360501289368
Class of detected object: 71
Name of detected objectsink
..............................
Box size of detected object[44, 122, 495, 269]
Confidence of detected object: 0.9923518896102905
Class of detected object: 71
Name of detected objectsink
..............................
Box size of detected object[38, 140, 508, 272]
Confidence of detected object: 0.9917967915534973
Class of detected object: 71
Name of detected objectsink
..............................
sink
sink,
==========================Analyusis of Image:D:/AmirWork/images/image8.jpg
height of image: 770
width of image: 478
Channels of image are: 3
Box size of detected object[81, 42, 269, 603]
Confidence of detected object: 0.6359930634498596
Class of detected object: 9
Name of detected objecttraffic light
..............................
Box size of detected object[116, 85, 240, 512]
Confidence of detected object: 0.9591655731201172
Class

Box size of detected object[87, 33, 448, 348]
Confidence of detected object: 0.9233090877532959
Class of detected object: 53
Name of detected objectpizza
..............................
pizza
pizza,
==========================Analyusis of Image:D:/AmirWork/images/image92.jpg
height of image: 506
width of image: 506
Channels of image are: 3
Box size of detected object[13, 33, 488, 457]
Confidence of detected object: 0.6970292925834656
Class of detected object: 53
Name of detected objectpizza
..............................
Box size of detected object[25, 49, 465, 456]
Confidence of detected object: 0.5528584718704224
Class of detected object: 53
Name of detected objectpizza
..............................
pizza
pizza,
==========================Analyusis of Image:D:/AmirWork/images/image93.jpg
height of image: 450
width of image: 567
Channels of image are: 3
Box size of detected object[29, 26, 529, 410]
Confidence of detected object: 0.7554365396499634
Class of detected object: 54
Name of de

Box size of detected object[142, 270, 248, 52]
Confidence of detected object: 0.7635560631752014
Class of detected object: 8
Name of detected objectboat
..............................
Box size of detected object[165, 269, 237, 53]
Confidence of detected object: 0.9693319797515869
Class of detected object: 8
Name of detected objectboat
..............................
Box size of detected object[174, 290, 13, 15]
Confidence of detected object: 0.7102283835411072
Class of detected object: 0
Name of detected objectperson
..............................
Box size of detected object[194, 287, 9, 17]
Confidence of detected object: 0.8132432699203491
Class of detected object: 0
Name of detected objectperson
..............................
Box size of detected object[213, 292, 10, 13]
Confidence of detected object: 0.6772352457046509
Class of detected object: 0
Name of detected objectperson
..............................
Box size of detected object[239, 287, 11, 17]
Confidence of detected object: 0

In [29]:
def GenerateSummary(text,rowRough):
    article_text = re.sub(r'\[[0-9]*\]', ' ', text)
    article_text = re.sub(r'\s+', ' ', article_text)
    formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )
    frmatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)
    sentence_list = nltk.sent_tokenize(article_text)
    word_frequencies = {}
    for word in nltk.word_tokenize(formatted_article_text):
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1
    maximum_frequncy = max(word_frequencies.values())
     
    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
    sentence_scores = {}
    for sent in sentence_list:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]
                #print(sentence_scores[sent])
    summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)
    summary = ' '.join(summary_sentences)
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(text,summary)
    print(scores)
    worksheetRough.write(rowRough, 2, str(summary))
    worksheetRough.write(rowRough, 3, str(scores))
    print("=================summary=============")
    print(summary)
    print("=================summary=============")

In [30]:
from nltk.corpus import sentiwordnet 
from rouge_score import rouge_scorer
stopwords = nltk.corpus.stopwords.words('english')
import heapq
workbook = xlsxwriter.Workbook('D:/AmirWork/OutPutImages/files/GeneratedDefinitions.xlsx')
worksheet1 = workbook.add_worksheet()
workbookRough = xlsxwriter.Workbook('D:/AmirWork/OutPutImages/files/SummaryRoughScore.xlsx')
worksheetRough = workbookRough.add_worksheet()
workbook = xlsxwriter.Workbook('D:/AmirWork/OutPutImages/files/GeneratedDefinitions.xlsx')
worksheet1 = workbook.add_worksheet()
workbookImageObject = xlsxwriter.Workbook('D:/AmirWork/OutPutImages/files/ImagesObjects.xlsx')
worksheetImageObject = workbookImageObject.add_worksheet()
worksheetImageObject.write(0, 0, "Image")
worksheetImageObject.write(0, 1, "Objects")
i=0
SummaryString=[]
row=0
rowRough=0
r=1
for img in imagesObjects:
    objects=img.split(',')
    sp=img.split(',')
    print(sp)
    res1 = list(set(sp) | set(sp))
    res1.sort()
    res1.remove("")
    print("Objects of Image-"+str(path + dataset_path[i]))
    worksheet1.write(row, 0, "Objects of Image-"+str(path + dataset_path[i]))
    worksheet1.write(row, 1, "Definitions")
    worksheetRough.write(rowRough, 0, "Objects of Image-"+str(path + dataset_path[i]))
    worksheetRough.write(rowRough, 1, "Definitions")
    worksheetRough.write(rowRough, 2, "Summary")
    worksheetRough.write(rowRough, 3, "RoughScore")
    worksheetImageObject.write(r, 0, str(dataset_path[i])[:-4])
    #worksheetImageObject.write(r, 3, img[:-1])
    #res=res.replace('"]', '')
    #res=res.replace(']"', '')
    #res.remove("']")
    #res.remove("['")
    worksheetImageObject.write(r, 1, str(res1))
    
    r=r+1
    row=row+1
    rowRough=rowRough+1
    for objct in objects:
        print(objct)
        worksheet1.write(row, 0, str(objct))
        worksheetRough.write(rowRough, 0, str(objct))
        sw=sentiwordnet.senti_synsets(objct)
        SummaryDef=""
        for w in sw:
            SummaryDef=SummaryDef+ w.synset.definition() + ".\n"
        print(SummaryDef)
        worksheet1.write(row, 1, str(SummaryDef))
        worksheetRough.write(rowRough, 1, SummaryDef)
        row=row+1
        if SummaryDef=="":
            SummaryDef="abc"
        GenerateSummary(SummaryDef,rowRough)
        rowRough=rowRough+1
        print("................")
    print("====================")
    i=i+1
workbook.close()
workbookRough.close()
workbookImageObject.close()

['vase', '']
Objects of Image-D:/AmirWork/images/image1.jpg
vase
an open jar of glass or porcelain used as an ornament or to hold flowers.

{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
=================summary=============
an open jar of glass or porcelain used as an ornament or to hold flowers.
=================summary=============
................


{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
=================summary=============
abc
=================summary=============
................
['person', 'boat', 'person', '']
Objects of Image-D:/AmirWork/images/image10.jpg
person
a human being.
a human body (usually including the clothing).
a grammatical category used in the classification of pronouns, possessive determiners, and verb forms according to whether they indicate the speaker, the addressee, or a third party.

{'rouge1': Score(precision

{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=0.7368421052631579, recall=0.7368421052631579, fmeasure=0.7368421052631579)}
=================summary=============
a motor vehicle with four wheels; usually propelled by an internal combustion engine. the compartment that is suspended from an airship and that carries personnel and the cargo and the power plant. a wheeled vehicle adapted to the rails of railroad. a conveyance for passengers or freight on a cable railway. where passengers ride up and down.
=================summary=============
................
truck
an automotive vehicle suitable for hauling.
a handcart that has a frame with two low wheels and a ledge at the bottom and handles at the top; used to move crates or other heavy objects.
convey (goods etc.) by truck.

{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=0.85, recall=0.85, fmeasure=0.85)}
=================summary=============
a handcart that has 

................
['truck', '']
Objects of Image-D:/AmirWork/images/image115.jpg
truck
an automotive vehicle suitable for hauling.
a handcart that has a frame with two low wheels and a ledge at the bottom and handles at the top; used to move crates or other heavy objects.
convey (goods etc.) by truck.

{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=0.85, recall=0.85, fmeasure=0.85)}
=================summary=============
a handcart that has a frame with two low wheels and a ledge at the bottom and handles at the top; used to move crates or other heavy objects. an automotive vehicle suitable for hauling. convey (goods etc.) by truck.
=================summary=============
................


{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
=================summary=============
abc
=================summary=============
................
['truck', '']
Objects of Image-D:/AmirWork/images/im

{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
=================summary=============
abc
=================summary=============
................
['stop sign', 'car', '']
Objects of Image-D:/AmirWork/images/image127.jpg
stop sign

{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
=================summary=============
abc
=================summary=============
................
car
a motor vehicle with four wheels; usually propelled by an internal combustion engine.
a wheeled vehicle adapted to the rails of railroad.
the compartment that is suspended from an airship and that carries personnel and the cargo and the power plant.
where passengers ride up and down.
a conveyance for passengers or freight on a cable railway.

{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=0.7368421052631579, recall=0.7368421052631579, fmeasu

{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=0.7297297297297297, recall=0.7297297297297297, fmeasure=0.7297297297297297)}
=================summary=============
a grammatical category used in the classification of pronouns, possessive determiners, and verb forms according to whether they indicate the speaker, the addressee, or a third party. a human body (usually including the clothing). a human being.
=================summary=============
................
stop sign

{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
=================summary=============
abc
=================summary=============
................
person
a human being.
a human body (usually including the clothing).
a grammatical category used in the classification of pronouns, possessive determiners, and verb forms according to whether they indicate the speaker, the addressee, or a third party.

{'rouge1': Score(preci

...

{'rouge1': Score(precision=1.0, recall=0.5458937198067633, fmeasure=0.7062499999999999), 'rougeL': Score(precision=0.5929203539823009, recall=0.32367149758454106, fmeasure=0.41875)}
=================summary=============
wheelwork consisting of a connected set of rotating gears by which force is transmitted or motion or torque is changed. public transport provided by a line of railway cars coupled together and drawn by a locomotive. piece of cloth forming the long back section of a gown that is drawn along the floor. undergo training or instruction in preparation for a particular role, function, or profession. a sequentially ordered set of things or events or ideas in which each successive member is related to the preceding. point or cause to go (blows, weapons, or objects such as photographic equipment) towards. develop (children's) behavior by instruction and practice; especially to teach self-control.
=================summary=============
................
person
a human being.
a huma

{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
=================summary=============
plant with dense clusters of tight green flower buds. branched green undeveloped flower heads.
=================summary=============
................
broccoli
plant with dense clusters of tight green flower buds.
branched green undeveloped flower heads.

{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
=================summary=============
plant with dense clusters of tight green flower buds. branched green undeveloped flower heads.
=================summary=============
................


{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
=================summary=============
abc
=================summary=============
................
['']
Objects of Image-D:/AmirWork/images/image50.jpg


{'rouge1': Sco

{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
=================summary=============
fruit with red or yellow or green skin and sweet to tart crisp whitish flesh. native Eurasian tree widely cultivated in many varieties for its firm rounded edible fruits.
=================summary=============
................
donut
a small ring-shaped friedcake.

{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
=================summary=============
a small ring-shaped friedcake.
=================summary=============
................
donut
a small ring-shaped friedcake.

{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
=================summary=============
a small ring-shaped friedcake.
=================summary=============
................
donut
a small ring-shaped friedcake.

{'rouge1': Score(precis

=================summary=============
a round vessel that is open at the top; used chiefly for holding food or liquids. a wooden ball (with flattened sides so that it rolls on a curved course) used in the game of lawn bowling. a large ball with finger holes used in the sport of bowling. a small round container that is open at the top for holding tobacco. a dish that is round and open at the top for serving foods. hurl a cricket ball from one end of the pitch towards the batsman at the other end. the act of rolling something (as the ball in bowling).
=================summary=============
................
broccoli
plant with dense clusters of tight green flower buds.
branched green undeveloped flower heads.

{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
=================summary=============
plant with dense clusters of tight green flower buds. branched green undeveloped flower heads.
=================summary==========

{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=0.7297297297297297, recall=0.7297297297297297, fmeasure=0.7297297297297297)}
=================summary=============
a grammatical category used in the classification of pronouns, possessive determiners, and verb forms according to whether they indicate the speaker, the addressee, or a third party. a human body (usually including the clothing). a human being.
=================summary=============
................
traffic light

{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
=================summary=============
abc
=================summary=============
................
car
a motor vehicle with four wheels; usually propelled by an internal combustion engine.
a wheeled vehicle adapted to the rails of railroad.
the compartment that is suspended from an airship and that carries personnel and the cargo and the power plant.
where passengers 

In [13]:
print("a")

a


In [31]:
import pandas as pd
InputData=pd.read_excel('D:\work\InputImageData.xlsx')
OutPutData=pd.read_excel('D:\work\OutPutImages\\files\\ImagesObjects.xlsx')



In [32]:
InputData

,S.No,Name,Object
0,1,Image1,vase
1,2,Image10,"boat, person"
2,3,Image100,boat
3,4,Image101,"boat, person"
4,5,Image102,"boat, person"
...,...,...,...
298,299,Image95,scissors
299,300,Image96,traffic light
300,301,Image97,"car, traffic light"
301,302,Image98,"car, person, traffic light"


In [33]:
InputData=InputData.sort_values("Name")

In [34]:
InputData=InputData.values

In [35]:
OutPutData=OutPutData.values

In [36]:
import xlsxwriter
workbookActualDetectedImageObjects = xlsxwriter.Workbook('D:/work/OutPutImages/files/DetectedImageObjects.xlsx')
worksheetActualDetectedImageObjects = workbookActualDetectedImageObjects.add_worksheet()
worksheetActualDetectedImageObjects.write(0, 0, "Actual Image Name")
worksheetActualDetectedImageObjects.write(0, 1, "Actual Objects")
worksheetActualDetectedImageObjects.write(0, 2, "Detected Image Name")
worksheetActualDetectedImageObjects.write(0, 3, "Detected Objects")
worksheetActualDetectedImageObjects.write(0, 4, "Detected? Yes or No")

0

In [37]:
row=1
actual=[]
for d in InputData:
    print(d[1])
    print(d[2])
    worksheetActualDetectedImageObjects.write(row, 0, d[1])
    worksheetActualDetectedImageObjects.write(row, 1, d[2])
    actual.append(d[2])
    row=row+1

Image1
vase
Image10
boat, person
Image100
boat
Image101
boat, person
Image102
boat, person
Image103
car, truck
Image104
boat
Image105
boat
Image106
truck
Image107
car, truck
Image108
truck
Image109
person, truck
Image11
traffic light
Image110
truck
Image111
truck
Image112
truck
Image113
truck
Image114
truck
Image115
truck
Image116
truck
Image117
truck
Image118
truck
Image119
truck
Image12
truck
Image120
truck
Image121
truck
Image122
truck
Image123
fire hydrant, person, stop sign
Image124
car, fire hydrant, stop sign
Image125
car, fire hydrant, person
Image126
fire hydrant, stop sign
Image127
car, stop sign
Image128
person, stop sign
Image129
fire hydrant, stop sign
Image13
traffic light
Image130
car, person, stop sign
Image131
fire hydrant, stop sign
Image132
fire hydrant, stop sign
Image133
fire hydrant
Image134
car, person, stop sign
Image135
person, stop sign
Image136
car, fire hydrant, stop sign
Image137
fire hydrant, person, stop sign
Image138
car, fire hydrant, stop sign
Image139

In [38]:
row=1
detected=[]
for d in OutPutData:
    d[1]=d[1].replace('[',"")
    d[1]=d[1].replace(']',"")
    d[1]=d[1].replace('\'',"")
    print(d[0])
    print(d[1])
    worksheetActualDetectedImageObjects.write(row, 2, d[0])
    worksheetActualDetectedImageObjects.write(row, 3, d[1])
    detected.append(d[1])
    row=row+1



image1
vase
image10
boat, person
image100
boat
image101
boat, person
image102
boat, person
image103
car, truck
image104

image105
boat
image106
truck
image107
car, truck
image108
truck
image109
person, truck
image11
traffic light
image110
truck
image111
truck
image112
truck
image113
truck
image114
truck
image115
truck
image116
truck
image117
truck
image118
truck
image119
truck
image12
truck
image120
truck
image121
truck
image122
truck
image123
fire hydrant, person, stop sign
image124
car, fire hydrant, stop sign
image125
car, fire hydrant, person
image126
fire hydrant, stop sign
image127
car, stop sign
image128
person, stop sign
image129
fire hydrant, stop sign
image13

image130
car, person, stop sign
image131
fire hydrant, stop sign
image132
fire hydrant, stop sign
image133
fire hydrant
image134
car, person, stop sign
image135
person, stop sign
image136
car, fire hydrant, stop sign
image137
fire hydrant, person, stop sign
image138
car, fire hydrant, stop sign
image139
fire hydrant, pe

In [39]:
from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [40]:
row=1
for i in range(0, len(OutPutData)):
    print(actual[i])
    print(detected[i])
    if actual[i]==detected[i]:
        print("1")
        worksheetActualDetectedImageObjects.write(row, 4, "1")
    else:
        print(similar(actual[i],detected[i]))
        worksheetActualDetectedImageObjects.write(row, 4, str(similar(actual[i],detected[i])))
    row=row+1
workbookActualDetectedImageObjects.close()

vase
vase
1
boat, person
boat, person
1
boat
boat
1
boat, person
boat, person
1
boat, person
boat, person
1
car, truck
car, truck
1
boat

0.0
boat
boat
1
truck
truck
1
car, truck
car, truck
1
truck
truck
1
person, truck
person, truck
1
traffic light
traffic light
1
truck
truck
1
truck
truck
1
truck
truck
1
truck
truck
1
truck
truck
1
truck
truck
1
truck
truck
1
truck
truck
1
truck
truck
1
truck
truck
1
truck
truck
1
truck
truck
1
truck
truck
1
truck
truck
1
fire hydrant, person, stop sign
fire hydrant, person, stop sign
1
car, fire hydrant, stop sign
car, fire hydrant, stop sign
1
car, fire hydrant, person
car, fire hydrant, person
1
fire hydrant, stop sign
fire hydrant, stop sign
1
car, stop sign
car, stop sign
1
person, stop sign
person, stop sign
1
fire hydrant, stop sign
fire hydrant, stop sign
1
traffic light

0.0
car, person, stop sign
car, person, stop sign
1
fire hydrant, stop sign
fire hydrant, stop sign
1
fire hydrant, stop sign
fire hydrant, stop sign
1
fire hydrant
fire hyd

In [44]:
import pandas as pd
pdf = pd.read_excel ("D:/work/OutPutImages/files/DetectedImageObjects.xlsx")
pdf = pdf[['Actual Objects','Detected Objects']]
pdf=pdf.values

In [45]:
pdf

array([['vase', 'vase'],
       ['boat, person', 'boat, person'],
       ['boat', 'boat'],
       ['boat, person', 'boat, person'],
       ['boat, person', 'boat, person'],
       ['car, truck', 'car, truck'],
       ['boat', 'No Object'],
       ['boat', 'boat'],
       ['truck', 'truck'],
       ['car, truck', 'car, truck'],
       ['truck', 'truck'],
       ['person, truck', 'person, truck'],
       ['traffic light', 'traffic light'],
       ['truck', 'truck'],
       ['truck', 'truck'],
       ['truck', 'truck'],
       ['truck', 'truck'],
       ['truck', 'truck'],
       ['truck', 'truck'],
       ['truck', 'truck'],
       ['truck', 'truck'],
       ['truck', 'truck'],
       ['truck', 'truck'],
       ['truck', 'truck'],
       ['truck', 'truck'],
       ['truck', 'truck'],
       ['truck', 'truck'],
       ['fire hydrant, person, stop sign',
        'fire hydrant, person, stop sign'],
       ['car, fire hydrant, stop sign', 'car, fire hydrant, stop sign'],
       ['car, fire h

In [46]:
actual=[]
detected=[]
i=1
for d in pdf:
    d[0]=d[0].replace('[',"")
    d[0]=d[0].replace(']',"")
    d[0]=d[0].replace('\'',"")
    print(d[0])
    actual.append(d[0])
    d[1]=d[1].replace('[',"")
    d[1]=d[1].replace(']',"")
    d[1]=d[1].replace('\'',"")
    print(d[1])
    detected.append(d[1])
    print(str(i)+"===========")
    i=i+1


vase
vase
1===========
boat, person
boat, person
2===========
boat
boat
3===========
boat, person
boat, person
4===========
boat, person
boat, person
5===========
car, truck
car, truck
6===========
boat
No Object
7===========
boat
boat
8===========
truck
truck
9===========
car, truck
car, truck
10===========
truck
truck
11===========
person, truck
person, truck
12===========
traffic light
traffic light
13===========
truck
truck
14===========
truck
truck
15===========
truck
truck
16===========
truck
truck
17===========
truck
truck
18===========
truck
truck
19===========
truck
truck
20===========
truck
truck
21===========
truck
truck
22===========
truck
truck
23===========
truck
truck
24===========
truck
truck
25===========
truck
truck
26===========
truck
truck
27===========
fire hydrant, person, stop sign
fire hydrant, person, stop sign
28===========
car, fire hydrant, stop sign
car, fire hydrant, stop sign
29===========
car, fire hydrant, person
car, fire hydrant, person
30===========


In [47]:
detected

['vase',
 'boat, person',
 'boat',
 'boat, person',
 'boat, person',
 'car, truck',
 'No Object',
 'boat',
 'truck',
 'car, truck',
 'truck',
 'person, truck',
 'traffic light',
 'truck',
 'truck',
 'truck',
 'truck',
 'truck',
 'truck',
 'truck',
 'truck',
 'truck',
 'truck',
 'truck',
 'truck',
 'truck',
 'truck',
 'fire hydrant, person, stop sign',
 'car, fire hydrant, stop sign',
 'car, fire hydrant, person',
 'fire hydrant, stop sign',
 'car, stop sign',
 'person, stop sign',
 'fire hydrant, stop sign',
 'No Object',
 'car, person, stop sign',
 'fire hydrant, stop sign',
 'fire hydrant, stop sign',
 'fire hydrant',
 'car, person, stop sign',
 'person, stop sign',
 'car, fire hydrant, stop sign',
 'fire hydrant, person, stop sign',
 'car, fire hydrant, stop sign',
 'fire hydrant, person, stop sign',
 'boat, person',
 'car, person',
 'horse, parking meter, person',
 'bench, person',
 'parking meter, person',
 'bench, parking meter',
 'parking meter',
 'car, parking meter',
 'parking

In [48]:
len(actual)

303

In [49]:

from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
# here we will use actual and predicted. predicted is UniqueImageObject and we 
#will make actual, both will be sorted
results = confusion_matrix(actual,detected) 
print ('Confusion Matrix :')
print(results) 
print ('Accuracy Score :',accuracy_score(actual,detected)) 
print ('Report : ')
print (classification_report(actual,detected)) 
cm=classification_report(actual,detected)


Confusion Matrix :
[[ 0  0  0 ...  0  0  0]
 [ 1 11  0 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]
 ...
 [ 0  0  0 ... 22  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 1  0  0 ...  0  0 13]]
Accuracy Score : 0.9042904290429042
Report : 
                                     precision    recall  f1-score   support

                          No Object       0.00      0.00      0.00         0
                          aeroplane       1.00      0.92      0.96        12
                       apple, donut       1.00      1.00      1.00         1
          backpack, bicycle, person       1.00      1.00      1.00         1
              backpack, bus, person       1.00      1.00      1.00         1
                              bench       1.00      1.00      1.00         1
                     bench, bicycle       1.00      1.00      1.00         1
                         bench, car       1.00      1.00      1.00         2
                 bench, car, person       1.00      1.00      1.00         1
        

C:\Users\saqib\anaconda3\envs\TF\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\saqib\anaconda3\envs\TF\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\saqib\anaconda3\envs\TF\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\saqib\anaconda3\envs\TF\lib\si